# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 29 2022 12:25PM,247542,15,30011061,"Alliance Pharma, Inc.",Released
1,Sep 29 2022 12:25PM,247542,15,30011062,"Alliance Pharma, Inc.",Released
2,Sep 29 2022 12:25PM,247542,15,30011064,"Alliance Pharma, Inc.",Released
3,Sep 29 2022 12:25PM,247542,15,30011065,"Alliance Pharma, Inc.",Released
4,Sep 29 2022 12:25PM,247542,15,30011063,"Alliance Pharma, Inc.",Released
5,Sep 29 2022 12:25PM,247542,15,30011066,"Alliance Pharma, Inc.",Released
6,Sep 29 2022 12:25PM,247542,15,30011068,"Alliance Pharma, Inc.",Released
7,Sep 29 2022 12:25PM,247542,15,30011067,"Alliance Pharma, Inc.",Released
8,Sep 29 2022 12:25PM,247542,15,30011071,"Alliance Pharma, Inc.",Released
9,Sep 29 2022 12:25PM,247542,15,30011070,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,247556,Executing,1
22,247558,Released,4
23,247559,Released,26
24,247563,Released,11
25,247564,Released,27


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247556,NaN,1.0,NaN
247558,NaN,NaN,4.0
247559,NaN,NaN,26.0
247563,NaN,NaN,11.0
247564,NaN,NaN,27.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247419,0.0,1.0,0.0
247421,0.0,1.0,0.0
247490,0.0,1.0,0.0
247509,10.0,0.0,1.0
247520,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247419,0,1,0
247421,0,1,0
247490,0,1,0
247509,10,0,1
247520,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247419,0,1,0
1,247421,0,1,0
2,247490,0,1,0
3,247509,10,0,1
4,247520,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247419,,1,
1,247421,,1,
2,247490,,1,
3,247509,10,,1
4,247520,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc."
41,Sep 29 2022 12:05PM,247564,10,"Methapharm, Inc."
68,Sep 29 2022 12:04PM,247563,10,"Methapharm, Inc."
79,Sep 29 2022 11:06AM,247559,10,Eywa Pharma Inc.
105,Sep 29 2022 10:56AM,247558,16,Sartorius Bioprocess Solutions
109,Sep 29 2022 10:46AM,247555,10,ISDIN Corporation
115,Sep 29 2022 10:43AM,247556,10,"Lupin Research, Inc."
116,Sep 29 2022 10:37AM,247544,10,Bio-PRF
128,Sep 29 2022 10:28AM,247552,16,TASA USA Inc.
129,Sep 29 2022 10:24AM,247553,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc.",,,41
1,Sep 29 2022 12:05PM,247564,10,"Methapharm, Inc.",,,27
2,Sep 29 2022 12:04PM,247563,10,"Methapharm, Inc.",,,11
3,Sep 29 2022 11:06AM,247559,10,Eywa Pharma Inc.,,,26
4,Sep 29 2022 10:56AM,247558,16,Sartorius Bioprocess Solutions,,,4
5,Sep 29 2022 10:46AM,247555,10,ISDIN Corporation,,3,3
6,Sep 29 2022 10:43AM,247556,10,"Lupin Research, Inc.",,1,
7,Sep 29 2022 10:37AM,247544,10,Bio-PRF,,11,1
8,Sep 29 2022 10:28AM,247552,16,TASA USA Inc.,,,1
9,Sep 29 2022 10:24AM,247553,10,MedStone Pharma LLC,,1,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc.",41,,
1,Sep 29 2022 12:05PM,247564,10,"Methapharm, Inc.",27,,
2,Sep 29 2022 12:04PM,247563,10,"Methapharm, Inc.",11,,
3,Sep 29 2022 11:06AM,247559,10,Eywa Pharma Inc.,26,,
4,Sep 29 2022 10:56AM,247558,16,Sartorius Bioprocess Solutions,4,,
5,Sep 29 2022 10:46AM,247555,10,ISDIN Corporation,3,3,
6,Sep 29 2022 10:43AM,247556,10,"Lupin Research, Inc.",,1,
7,Sep 29 2022 10:37AM,247544,10,Bio-PRF,1,11,
8,Sep 29 2022 10:28AM,247552,16,TASA USA Inc.,1,,
9,Sep 29 2022 10:24AM,247553,10,MedStone Pharma LLC,8,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc.",41,,
1,Sep 29 2022 12:05PM,247564,10,"Methapharm, Inc.",27,,
2,Sep 29 2022 12:04PM,247563,10,"Methapharm, Inc.",11,,
3,Sep 29 2022 11:06AM,247559,10,Eywa Pharma Inc.,26,,
4,Sep 29 2022 10:56AM,247558,16,Sartorius Bioprocess Solutions,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc.",41.0,NaN,NaN
1,Sep 29 2022 12:05PM,247564,10,"Methapharm, Inc.",27.0,NaN,NaN
2,Sep 29 2022 12:04PM,247563,10,"Methapharm, Inc.",11.0,NaN,NaN
3,Sep 29 2022 11:06AM,247559,10,Eywa Pharma Inc.,26.0,NaN,NaN
4,Sep 29 2022 10:56AM,247558,16,Sartorius Bioprocess Solutions,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 12:25PM,247542,15,"Alliance Pharma, Inc.",41.0,0.0,0.0
1,Sep 29 2022 12:05PM,247564,10,"Methapharm, Inc.",27.0,0.0,0.0
2,Sep 29 2022 12:04PM,247563,10,"Methapharm, Inc.",11.0,0.0,0.0
3,Sep 29 2022 11:06AM,247559,10,Eywa Pharma Inc.,26.0,0.0,0.0
4,Sep 29 2022 10:56AM,247558,16,Sartorius Bioprocess Solutions,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2475458,82.0,18.0,20.0
15,247542,41.0,0.0,0.0
16,990195,7.0,2.0,0.0
20,247520,0.0,1.0,0.0
21,742330,0.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2475458,82.0,18.0,20.0
1,15,247542,41.0,0.0,0.0
2,16,990195,7.0,2.0,0.0
3,20,247520,0.0,1.0,0.0
4,21,742330,0.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,82.0,18.0,20.0
1,15,41.0,0.0,0.0
2,16,7.0,2.0,0.0
3,20,0.0,1.0,0.0
4,21,0.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,82.0
1,15,Released,41.0
2,16,Released,7.0
3,20,Released,0.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,20,21
Status,,,,,
Completed,20.0,0.0,0.0,0.0,0.0
Executing,18.0,0.0,2.0,1.0,3.0
Released,82.0,41.0,7.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,20,21
0,Completed,20.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,2.0,1.0,3.0
2,Released,82.0,41.0,7.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,20,21
0,Completed,20.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,2.0,1.0,3.0
2,Released,82.0,41.0,7.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()